In [2]:
import pathlib

import pymeshlab
from pymeshlab import AbsoluteValue, MeshSet

from cfdmod.api.geometry.STL import export_stl, read_stl


def apply_remeshing(element_size: float, mesh_path: pathlib.Path, output_path: pathlib.Path):
    ms: MeshSet = pymeshlab.MeshSet()
    ms.load_new_mesh(mesh_path)
    ms.meshing_isotropic_explicit_remeshing(
        iterations=15, targetlen=AbsoluteValue(element_size), selectedonly=True
    )
    ms.save_current_mesh(output_path, binary=False)


def translate_mesh(
    translation: tuple[float, float, float],
    mesh_path: pathlib.Path,
    output_path: pathlib.Path,
):
    triangles, normals = read_stl(mesh_path)
    triangles[:, :, 0] += translation[0]
    triangles[:, :, 1] += translation[1]
    triangles[:, :, 2] += translation[2]
    export_stl(output_path, triangles, normals)

In [41]:
import numpy as np

def slice_surface(plane_normal: tuple[float, float, float], plane_origin: tuple[float, float, float], mesh_path: pathlib.Path):
    distance = np.dot(plane_normal, plane_origin) / np.linalg.norm(plane_normal)
    ms: MeshSet = pymeshlab.MeshSet()
    ms.load_new_mesh(str(mesh_path.absolute()))
    ms.generate_polyline_from_planar_section(
        planeaxis = "Custom Axis", 
        customaxis = plane_normal,
        planeoffset = distance,
        relativeto = "Origin", # 'Bounding box center' 'Bounding box min'
        splitsurfacewithsection=True
    )

    for index in [2,3]:
        output_path=mesh_path.parent / f"{mesh_path.name}-{index-2}.stl"
        ms.set_current_mesh(index)
        ms.save_current_mesh(str(output_path.absolute()))
    
mesh_path = pathlib.Path("./fixtures/tests/notebooks/000.stl")
slice_surface(plane_normal = [0,1,1], plane_origin=[0,10,10], mesh_path=mesh_path)

14.14213562373095


### Example usage

In [ ]:
folder_path = pathlib.Path("./fixtures/tests/notebooks")
mesh_list = ["sfc1", "sfc2"]

for mesh_name in mesh_list:
    slice_surface(plane_normal = [0,1,1], plane_origin=[0,10,10], mesh_path=folder_path / mesh_name)

In [44]:
folder_path = pathlib.Path("./fixtures/tests/notebooks")

for mesh_path in folder_path.iterdir():
    if not mesh_path.is_file():
        continue
    slice_surface(plane_normal = [0,1,1], plane_origin=[0,10,10], mesh_path=folder_path / mesh_name)

fixtures\tests\notebooks\000.stl
fixtures\tests\notebooks\000.stl-0.stl
fixtures\tests\notebooks\000.stl-1.stl
